<a href="https://colab.research.google.com/github/darshanvjani/pytorch-lightning/blob/main/5.%20data%20module/data_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pytorch_lightning
!pip install -U torch_xla
!pip install cloud-tpu-client

In [19]:
# import torch_xla as xla

In [20]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader,random_split
from tqdm import tqdm

import pytorch_lightning as pl
import torchmetrics
from torchmetrics import Metric

In [25]:
from torch.utils import data
# from torch.nn.parallel.data_parallel import device

class MyAccuracy(Metric):
  def __init__(self, num_classes=10):
    super().__init__()
    self.num_classes = num_classes
    self.add_state("correct", default=torch.tensor(0), dist_reduce_fx="sum")
    self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")

  def update(self, preds, target):
    preds = torch.argmax(preds, dim=1)
    assert preds.shape == target.shape
    self.correct += (preds == target).sum()
    self.total += target.numel()

  def compute(self):
    return self.correct.float() / self.total.float()


class NN(pl.LightningModule):
  def __init__(self, input_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size,50)
    self.fc2 = nn.Linear(50,num_classes)
    self.loss_fc = nn.CrossEntropyLoss()
    # self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
    self.accuracy = MyAccuracy(num_classes)
    self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

  def _common_step(self, batch, batch_idx):
    x, y = batch
    x = x.reshape(x.size(0),-1)
    y_pred = self.forward(x)  # forward pass
    loss = self.loss_fc(y_pred, y)  # compute loss
    return loss, y_pred, y

  def training_step(self, batch, batch_idx):
    loss, y_pred, y = self._common_step(batch, batch_idx)
    accuracy = self.accuracy(y_pred, y)
    f1_score = self.f1_score(y_pred, y)
    self.log_dict({"train_loss": loss, "train_acc": accuracy, "train_f1": f1_score}, prog_bar=True, on_step=False, on_epoch=True)
    return loss

  def test_step(self, batch, batch_idx):
    loss, y_pred, y = self._common_step(batch, batch_idx)
    self.log('test_loss', loss)
    return loss

  def validation_step(self, batch, batch_idx):
    loss, y_pred, y = self._common_step(batch, batch_idx)
    self.log('val_loss', loss)
    return loss

  def predict_step(self, batch, batch_idx):
    x, y = batch
    x = x.reshape(-1, 1)
    y_pred_final = self.forward(x)
    prediction = torch.argmax(y_pred_final, dim=1)
    return prediction

  def configure_optimizers(self):
    return optim.Adam(self.parameters(), lr=0.001)

class MNISTDataModule(pl.LightningDataModule):
  def __init__(self, data_dir, batch_size, num_workers):
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
    self.num_workers = num_workers

  def prepare_data(self):
    datasets.MNIST(self.data_dir, train=True, download=True)
    datasets.MNIST(self.data_dir, train=False, download=True)

  def setup(self, stage: str):
    entire_dataset = datasets.MNIST(
      root=self.data_dir,
      train=True,
      transform=transforms.ToTensor(),
      download=False
    )

    self.train_ds, self.val_ds = random_split(entire_dataset, [50000, 10000])
    self.test_ds = datasets.MNIST(
      root=self.data_dir,
      train=False,
      transform=transforms.ToTensor(),
      download=False
    )

  def train_dataloader(self):
    return DataLoader(
      self.train_ds,
      batch_size=self.batch_size,
      num_workers=self.num_workers,
      shuffle=True
    )
  def test_dataloader(self):
    return DataLoader(
      self.test_ds,
      batch_size=self.batch_size,
      num_workers=self.num_workers,
      shuffle=False
    )

  def val_dataloader(self):
    return DataLoader(
      self.val_ds,
      batch_size=self.batch_size,
      num_workers=self.num_workers,
      shuffle=False
    )


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#HP

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3


# Load Data

dm = MNISTDataModule(
  data_dir="dataset/",
  batch_size=batch_size,
  num_workers=4
)

# entire_dataset = datasets.MNIST(
#     root="dataset/", train=True, transform=transforms.ToTensor(), download=True
# )
# train_ds, val_ds = random_split(entire_dataset, [50000, 10000])
# test_ds = datasets.MNIST(
#     root="dataset/", train=False, transform=transforms.ToTensor(), download=True
# )
# train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(dataset=val_ds, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False)

In [26]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [ ]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [27]:
trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=3, precision=16)
trainer.fit(model, dm)
trainer.validate(model, dm)
trainer.test(model, dm)

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type              | Params
-----------------------------------------------
0 | fc1      | Linear            | 39.2 K
1 | fc2      | Linear            | 510   
2 | loss_fc  | CrossEntropyLoss  | 0     
3 | accuracy | MyAcc

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.1774793267250061     │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.1628066748380661     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.1628066748380661}]